In [ ]:

# Snow Water Equivalent (SWE): Measures the amount of water contained in the snowpack.
# Snow Depth: The depth of the snowpack.
# Snow Melt Runoff: The amount of meltwater running off at the base of the snowpack.
# Sublimation from the Snow Pack: The process where snow changes directly from solid to gas.
# Sublimation of Blowing Snow: Sublimation specific to snow that's being transported by wind.
# Solid Precipitation: Snowfall or other forms of solid precipitation.
# Liquid Precipitation: Rainfall or other forms of liquid precipitation.
# Snow Pack Average Temperature: The average temperature within the snowpack.

import arcpy
from arcpy.sa import *
import tarfile
import gzip
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import glob
from datetime import datetime
from tqdm import tqdm
from osgeo import gdal, osr
import geopandas as gpd
import rasterio
def reading_geoTIFF_metafile(meta_data_path,file_path):
    geoTIFF_metadata = {}
    with open(meta_data_path, 'r') as f:
        meta = f.readlines()
        for line in meta:
            if 'Description' in line:
                geoTIFF_metadata['variable_name'] = line.split(':')[1]
            if 'Number of columns' in line:
                geoTIFF_metadata['num_columns'] = int(line.split(':')[1].strip())
            elif 'Number of rows' in line:
                geoTIFF_metadata['num_rows'] = int(line.split(':')[1].strip())
            elif 'No data value' in line:
                geoTIFF_metadata['no_data_value'] = float(line.split(':')[1].strip())
            elif 'Minimum x-axis coordinate' in line:
                geoTIFF_metadata['x_min'] = float(line.split(':')[1].strip())
            elif 'Maximum y-axis coordinate' in line:
                geoTIFF_metadata['y_max'] = float(line.split(':')[1].strip())
            elif 'X-axis resolution' in line:
                geoTIFF_metadata['x_res'] = float(line.split(':')[1].strip())
            elif 'Y-axis resolution' in line:
                geoTIFF_metadata['y_res'] = -float(line.split(':')[1].strip())  
            elif 'X-axis offset' in line:
                geoTIFF_metadata['x_skew'] = float(line.split(':')[1].strip())
            elif 'Y-axis offset' in line:
                geoTIFF_metadata['y_skew'] = float(line.split(':')[1].strip())
            
            elif 'Start year' in line:
                geoTIFF_metadata['year'] = int(line.split(':')[1].strip())
            elif 'Start month' in line:
                geoTIFF_metadata['month'] = int(line.split(':')[1].strip())
            elif 'Start day' in line:
                geoTIFF_metadata['day'] = int(line.split(':')[1].strip())


            elif 'Minimum data value' in line:
                geoTIFF_metadata['min'] = float(line.split(':')[1].strip())
            elif 'Maximum data value' in line:
                geoTIFF_metadata['max'] = float(line.split(':')[1].strip())
            elif 'No data value' in line:
                geoTIFF_metadata['novalue'] = int(line.split(':')[1].strip())

    return geoTIFF_metadata
    
def create_raster(data, metadata, output_path):
    """
    Create a raster file from the provided data and metadata.
    """
    # Create a driver to write the raster file
    driver = gdal.GetDriverByName('GTiff')
#    print( data.shape)
    out_raster = driver.Create(output_path, data.shape[1], data.shape[0], 1, gdal.GDT_UInt16)

    
    # Set geotransform and projection
    geotransform = [metadata['x_min'], metadata['x_res'], metadata['x_skew'],
                    metadata['y_max'], metadata['y_skew'], metadata['y_res']]

    out_raster.SetGeoTransform(geotransform)    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # WGS84
    out_raster.SetProjection(srs.ExportToWkt())

    # Write data to the raster band
    out_band = out_raster.GetRasterBand(1)
    out_band.WriteArray(data)
    out_band.SetNoDataValue(metadata['no_data_value'])
    out_band.FlushCache()

    # Close the dataset
    out_raster = None


def create_plot(data_2d, variable_name):
                plt.imshow(data_2d, cmap='gray')
                variable_name = variable_name.split(',')[0]
                variable_name = variable_name.replace(' ','_')    
                plt.title(variable_name)
                plt.colorbar()
                plt.show()

def open_gz_files(base_directory, target_dir):
    file_paths = glob.glob(base_directory + '*.gz')
    os.makedirs(target_dir, exist_ok=True)
    for file_path in file_paths:
        new_file_path = os.path.join(target_dir, os.path.basename(file_path[:-3]))
        with gzip.open(file_path, 'rb') as f_in:
            with open(new_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(file_path)


        
def remove_dat_txt_file (target_dir):
    files= os.listdir(target_dir)
    for file in files:
        if '.dat' in file or '.txt' in file or '.bil' in file or '.hdr' in file:
            os.remove(os.path.join(target_dir, file))



# Function to generate a continuous date range
def generate_date_range(start_date, end_date):
    return [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]

def remove_dat_txt_file (target_dir):
    files= os.listdir(target_dir)
    for file in files:
        if '.dat' in file or '.txt' in file or '.bil' in file or '.hdr' in file:
          #  os.remove(os.path.join(target_dir, file))
            arcpy.Delete_management(os.path.join(target_dir, file))



# Main processing
tar_file_paths = glob.glob(r'/data/MyDataBase/SWATGenXAppData/snow/snow/' + '*.tar')
print(f'Number of tar files: {len(tar_file_paths)}')

entire_data = {}
dates_processed = set()


def rename_dat_to_bil(target_dir, file, file_path):

    new_file_path = os.path.join(target_dir, file[:-4] + ".bil")
    
    os.rename(file_path, new_file_path)



for tar_file_path in tqdm(tar_file_paths, desc='Processing tar files'):
    # Your existing code for processing each tar file

    with tarfile.open(tar_file_path, "r:") as tar:
        tar.extractall(path=os.path.dirname(tar_file_path))

    base_directory = '/data/MyDataBase/SWATGenXAppData/snow/snow/'
    subset_name = 'michigan'
    target_dir = os.path.join(base_directory, subset_name)
    open_gz_files(base_directory, target_dir)

    for file in os.listdir(target_dir):
        if file.endswith(".dat"):
         #   print(file)
            file_path = os.path.join(target_dir,file)
            rename_dat_to_bil(target_dir,file,file_path)
            
            bil_file_path = file_path[:-4] + '.bil'
            meta_data_path = file_path[:-4] + '.hdr'
            
            with open(meta_data_path, 'w') as f:
                f.write('byteorder M\nlayout bil\n nbands 1\n nbits 16 \n ncols 6935\n nrows 3351\n ulxmap -124.729583333331703\n ulymap 52.871249516804028\n xdim 0.0083333333\n ydim 0.00833333333\n' 
)
                
            meta_data_path = file_path[:-4] + '.txt'
            geoTIFF_metadata = reading_geoTIFF_metafile(meta_data_path,file_path)


            day = geoTIFF_metadata['day']
            month = geoTIFF_metadata['month']
            year = geoTIFF_metadata['year']      

            variable_name = geoTIFF_metadata['variable_name'].split(',')[0].replace('-', '_').replace(' ', '_').replace('\n', '')

            
            out_geotif_path = os.path.join(os.path.dirname(file_path),f'{year}',f'{month}',f'{day}',f'{variable_name}.tif')
            os.makedirs(os.path.dirname(out_geotif_path), exist_ok=True)


            arcpy.env.overwriteOutput=True
            clipped_raster_path = out_geotif_path
            
            # Michigan extent (example values, replace with actual extents)
            x_min, y_min, x_max, y_max = -87, 41.5, -82, 47
            rectangle = f"{x_min} {y_min} {x_max} {y_max}"
            
            # Open the .bil file as a raster
            bil_raster = arcpy.Raster(bil_file_path)
            
            min_value = geoTIFF_metadata['min'] 
            max_value =geoTIFF_metadata['max'] 

           
 
            arcpy.Clip_management(bil_raster, rectangle, clipped_raster_path)
            
            # Read the clipped raster
            clipped_raster = Raster(clipped_raster_path)
            
            # Apply minimum and maximum constraints
            constrained_raster = Con((clipped_raster < min_value) | (clipped_raster > max_value), np.nan, clipped_raster)
            
            # Save the constrained raster
            constrained_raster_path = clipped_raster_path.replace(".tif", "_constrained.tif")
            
            constrained_raster.save(constrained_raster_path)
            arcpy.Delete_management(clipped_raster_path)
            
    remove_dat_txt_file(target_dir)
    

In [ ]:

import rasterio
import numpy as np
import os
from calendar import monthrange
# Initialize an empty list for Snow Melt Rate (SMR)

def storing_rasters(start_year, end_year, name_of_dataset):
    smr = []
    for year in range(start_year, end_year+1):
        print(year)
        lat_long = 0
        for month in range(1, 13):
            days_in_month = monthrange(year, month)[1]
            for day in range(1, days_in_month + 1):
                file_path = fr'/data/MyDataBase/SWATGenXAppData/snow/snow/michigan/{year}/{month}/{day}/{name_of_dataset}'
                
                # Check if the file exists
                if not os.path.exists(file_path):
                    data = np.nan * np.zeros([660, 600]) 
                else:
                    # Open the file
                    with rasterio.open(file_path) as src:
                        # Read the raster values
                        data = src.read(1)
                      #  print(data.shape)
                        data = np.where(abs(data) > 1e12, np.nan, data)
            
                        if lat_long==0:
                            lat_long = 1
                            transform = src.transform
                            latitudes = np.zeros(data.shape, dtype=np.float64)
                            longitudes = np.zeros(data.shape, dtype=np.float64)
            
                            # Calculate latitude and longitude for each pixel
                            for row in range(data.shape[0]):
                                for col in range(data.shape[1]):
                                    x, y = rasterio.transform.xy(transform, row, col, offset='center')
                                    longitudes[row, col] = x
                                    latitudes[row, col] = y
                                
                smr.append(data)

    smr = np.stack(smr, axis=0)
    return smr, longitudes, latitudes



import netCDF4 as nc


def create_netcdf(stacked_data, lons, lats, start_year, end_year,name_of_dataset):
    filename=f'D:\MyDataBase\snow\SNODAS{name_of_dataset[:-4]}_{start_year}_{end_year}.nc'
    # Determine the dimensions
    time, lat, lon = stacked_data.shape

    # Create a new NetCDF file
    dataset = nc.Dataset(filename, 'w', format='NETCDF4_CLASSIC')

    # Create dimensions - time, latitude, longitude
    dataset.createDimension('time', time)
    dataset.createDimension('lat', lat)
    dataset.createDimension('lon', lon)

    # Create variables
    times = dataset.createVariable('time', 'i4', ('time',))
    latitudes = dataset.createVariable('lat', 'f4', ('lat',))
    longitudes = dataset.createVariable('lon', 'f4', ('lon',))
    values = dataset.createVariable('value', 'f4', ('time', 'lat', 'lon',))

    # Define units and other attributes
    times.units = 'days since ' + str(start_year) + '-01-01'
    latitudes.units = 'degrees_north'
    longitudes.units = 'degrees_east'
    values.units = 'Unknown'  # Replace with the actual unit of your data

    # Assign data to the variables
    times[:] = np.arange(time)  # Assuming each time step is one day
    
    # Flatten the latitudes and longitudes if they are 2D
    if lats.ndim > 1:
        latitudes[:] = lats[:, 0]  # Assuming latitudes are constant across rows
    else:
        latitudes[:] = lats

    if lons.ndim > 1:
        longitudes[:] = lons[0, :]  # Assuming longitudes are constant across columns
    else:
        longitudes[:] = lons

    values[:, :, :] = stacked_data

    # Close the NetCDF file
    dataset.close()

name_of_datasets = [   '_Modeled_melt_rate_constrained.tif',
    #'_Modeled_average_temperature_constrained.tif', '_Modeled_blowing_snow_sublimation_rate_constrained.tif', '_Modeled_snow_layer_thickness_constrained.tif', '_Modeled_snow_water_equivalent_constrained.tif', 
   # '_Modeled_snowpack_sublimation_rate_constrained.tif', '_Non_snow_accumulation_constrained.tif', '_Snow_accumulation_constrained.tif' 
                   ]  #'_Modeled_melt_rate_constrained.tif',

np.savetxt(fname='/data/MyDataBase/SWATGenXAppData/snow/SNODAS_latitudes_michigan.txt', X = latitudes)
np.savetxt(fname='/data/MyDataBase/SWATGenXAppData/snow/SNODAS_longitudes_michigan.txt', X = longitudes)

start_year = 2004
end_year = 2023
for name_of_dataset in name_of_datasets:
    print(name_of_dataset)
    stacked_dataset, longitudes, latitudes  = storing_rasters(start_year, end_year, name_of_dataset)  
    create_netcdf(stacked_dataset, longitudes, latitudes, start_year, end_year, name_of_dataset)


########################### creating location shapeifles

from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import numpy as np
latitudes = np.loadtxt('/data/MyDataBase/SWATGenXAppData/snow/SNODAS_latitudes_michigan.txt')                    
longitudes = np.loadtxt('/data/MyDataBase/SWATGenXAppData/snow/SNODAS_longitudes_michigan.txt')
latitudes_shape = latitudes.shape  # Assuming latitudes is a 2D NumPy array
longitudes_shape = longitudes.shape  # Assuming longitudes is a 2D NumPy array

# Flatten the arrays and also get the corresponding row and column indices
lat_flat = latitudes.flatten()
long_flat = longitudes.flatten()
row_indices, col_indices = np.indices(latitudes_shape)

# Create DataFrame
SNODAS_stations = pd.DataFrame({
    'geometry': [Point(x, y) for x, y in zip(long_flat, lat_flat)],
    'lat': lat_flat,
    'long': long_flat,
    'row_id': row_indices.flatten(),
    'col_id': col_indices.flatten(),
    'ID': np.arange(100000, 100000 + lat_flat.size)
})

# Convert to GeoDataFrame
SNODAS_stations = gpd.GeoDataFrame(SNODAS_stations, crs='EPSG:4326', geometry='geometry')


# Save to shapefile
SNODAS_stations.to_file(fr'D:\MyDataBase\snow\SNODAS_locations.shp')

In [ ]:
import os
import geopandas as gpd
import numpy as np
import concurrent.futures
import netCDF4 as nc
import pandas as pd

def load_variable(file_path, var_name):
    with nc.Dataset(file_path) as dataset:
        return dataset.variables[var_name][:]

def loading_dataset(BASE_PATH):
    # Load latitudes and longitudes
    SNODAS_stations=gpd.read_file(os.path.join(BASE_PATH, 'snow/SNODAS_locations.shp'))
    latitudes  = np.loadtxt(os.path.join(BASE_PATH,'snow/SNODAS_latitudes_michigan.txt'))                   
    longitudes = np.loadtxt(os.path.join(BASE_PATH,'snow/SNODAS_longitudes_michigan.txt'))    
    # Define file paths and variable names
    datasets = {
        
        'average_temperature':             os.path.join(BASE_PATH, 'snow/SNODAS_Modeled_average_temperature_constrained_2004_2023.nc'),
        'blowing_snow_sublimation_rate':   os.path.join(BASE_PATH, 'snow/SNODAS_Modeled_blowing_snow_sublimation_rate_constrained_2004_2023.nc'),
        'melt_rate':                       os.path.join(BASE_PATH, 'snow/SNODAS_Modeled_melt_rate_constrained_2004_2023.nc'),
        'snow_layer_thickness':            os.path.join(BASE_PATH, 'snow/SNODAS_Modeled_snow_layer_thickness_constrained_2004_2023.nc'),
        'snow_water_equivalent':           os.path.join(BASE_PATH, 'snow/SNODAS_Modeled_snow_water_equivalent_constrained_2004_2023.nc'),
        'snowpack_sublimation_rate':       os.path.join(BASE_PATH, 'snow/SNODAS_Modeled_snowpack_sublimation_rate_constrained_2004_2023.nc'),
        'non_snow_accumulation':           os.path.join(BASE_PATH, 'snow/SNODAS_Non_snow_accumulation_constrained_2004_2023.nc'),
        'snow_accumulation':               os.path.join(BASE_PATH, 'snow/SNODAS_Snow_accumulation_constrained_2004_2023.nc')
    }
    
    # Load data from NetCDF files
    variables = {key: load_variable(path, 'value') for key, path in datasets.items()}

    return variables, longitudes, latitudes, SNODAS_stations, datasets

BASE_PATH = f'D:/MyDataBase'

variables, longitudes, latitudes , SNODAS_stations, datasets = loading_dataset(BASE_PATH)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


variable_names = [ 'average_temperature',             
        'melt_rate',               
        'snow_layer_thickness',           
            'snow_water_equivalent',          
        'snowpack_sublimation_rate',      
        'non_snow_accumulation',       
        'snow_accumulation']

converters = [1,
        1/100,
        1/1,
        1/1,
        1/ 100,
        1/10,
        1/10]

units = ['Kelvin',
'mm',
'mm',
'mm',
'mm',
'mm',
'kg/sqm',
'kg/sqm']
              
for variable_name,converter, unit in zip(variable_names, converters, units): 
    # Assuming 'melt_rate' is a 3D array with shape (days, height, width)
    melt_rate = converter*variables[variable_name][:]  # Example array

    # Define the start year and the total number of days
    start_year = 2000  # Example start year
    num_days = melt_rate.shape[0]

    # Create a date range
    dates = pd.date_range(start=datetime(start_year, 1, 1), periods=num_days)

    # Calculate the number of years
    num_years = len(np.unique(dates.year))

    # Reshape melt_rate to have years, months, and days
    annual_melt_rate = np.empty((num_years, 12, *melt_rate.shape[1:]))

    # Calculate monthly mean melt rate
    for year in range(num_years):
        for month in range(1, 13):
            # Select data for the current month and year
            mask = (dates.year == start_year + year) & (dates.month == month)
            monthly_data = melt_rate[mask, :, :]

            # Check if there are any non-NaN values before calculating the mean
            if np.any(~np.isnan(monthly_data)):
                # Calculate the mean, ignoring NaN values
                monthly_mean = np.nanmean(monthly_data, axis=0)
            else:
                # If all values are NaN, fill the array segment with NaNs
                monthly_mean = np.full(melt_rate.shape[1:], np.nan)

            annual_melt_rate[year, month-1, :, :] = monthly_mean

    # Calculate annual mean melt rate
    with np.errstate(invalid='ignore'):
        annual_mean_melt_rate = np.nanmean(annual_melt_rate, axis=(0, 1))

    # Plotting the monthly mean for each month across all years
    fig, axs = plt.subplots(3, 4, figsize=(15, 10))  # Adjust the size as needed
    axs = axs.flatten()
    month_label = ['Jan','Feb','March','April','May','June','July','Agu','Sep','Oct','Nov','Dec']

    for month in range(12):
        # Only plot if the monthly mean contains non-NaN values
        if not np.all(np.isnan(annual_melt_rate[:, month, :, :])):
            avm=np.nanmean(annual_melt_rate[:, month, :, :], axis=0)
            im = axs[month].imshow(avm, cmap='winter_r', vmax=np.nanpercentile(avm, 97.5), vmin=np.nanpercentile(avm, 2.5))
            axs[month].set_title(f'Mean. {variable_name}({unit}):{month_label[month]}')
            fig.colorbar(im, ax=axs[month])
        else:
            # If the entire month has no data, display a message
            axs[month].text(0.5, 0.5, 'No data for this month', horizontalalignment='center', verticalalignment='center', transform=axs[month].transAxes)
            axs[month].set_title(f'Mean. Monthly: {month_label[month]}')

    plt.tight_layout()
    plt.savefig(f'D:\MyDataBase\Documentations\SNODAS_annual_monthly_figures\Average_Monthly_{variable_name}_Michigan_SNODAS.jpeg',dpi=300)
    plt.title(f'Montly {variable_name}({unit})')
    plt.show()

    # Plotting the annual mean across all months and years
    plt.figure(figsize=(10, 10))  # Adjust the size as needed
    im = plt.imshow(annual_mean_melt_rate, cmap='winter_r', vmax=np.nanpercentile(annual_mean_melt_rate,97.25), vmin=np.nanpercentile(annual_mean_melt_rate,2.5))
    plt.title(f'Mean Annual {variable_name}({unit})')
    plt.colorbar(im)
    plt.savefig(f'/data/MyDataBase/SWATGenXAppData/Documentations/SNODAS_annual_monthly_figures/Average_Annual_{variable_name}_michigan_SNODAS.jpeg',dpi=300)
    plt.show()


In [ ]:
import datetime
import os
from shapely import Point
import shutil

def saving_output(output,out_file_path):
    
    output = output.rename(columns = {
        'average_temperature': 'SAT',              #  units: Kelvin                                Description: Modeled average temperature, SWE-weighted average of snow layers  
        'blowing_snow_sublimation_rate': 'BSSR',   #  units: Meters / 100000                       Description: Modeled blowing snow sublimation rate, 24-hour total         ### convert it to mm  by multiplying with 1/100
        'melt_rate': 'MR',                         #  units: Meters / 100000                       Description: Modeled melt rate, bottom of snow layers, 24-hour total      ### convert it to mm  by multiplying with 1/100
        'snow_layer_thickness': 'SLT',             #  units: Meters / 1000                         Description: Modeled snow layer thickness, total of snow layer            ### convert it to mm  by multiplying with 1/1
        'snow_water_equivalent': 'SWE',            #  units: Meters / 1000                         Description: Modeled snow water equivalent, total of snow layers          ### convert it to mm  by multiplying with 1/1
        'snowpack_sublimation_rate': 'SSR',        #  units: Meters / 100000                       Description: Modeled snowpack sublimation rate, 24-hour total             ### convert it to mm  by multiplying with 1/ 100
        'non_snow_accumulation': 'NSA',            #  units: Kilograms per square meter / 10       Description: Non-snow accumulation, 24-hour total                         ### convert it to kg/sqm by multiplying with 1/10
        'snow_accumulation': 'SA'                  #  units: Kilograms per square meter / 10       Description: Snow accumulation, 24-hour total                             ### convert it to kg/sqm by multiplying with 1/10
    })
    
    output['year'] = output.date.dt.year
    output['day'] = output.date.dt.dayofyear    #### i want to get the day based on 0 to 365
    # Append the DataFrame without header
    output = output[['year','day','SA','NSA','MR','SWE', 'BSSR','SLT','SSR','SAT']]
    
    # Unit conversions
    output['SAT']  = output['SAT']   -273.15             # Kelvin to celsius degree
    output['BSSR'] = output['BSSR']  * 1000/100000       # Meters/100000 to mm
    output['MR']   = output['MR']    * 1000/100000       # Meters/100000 to mm
    output['SLT']  = output['SLT']   * 1000/1000         # Meters/1000 to mm
    output['SWE']  = output['SWE']   * 1000/1000         # Meters/1000 to mm
    output['SSR']  = output['SSR']   * 1000/100000       # Meters/100000 to mm
    output['NSA']  = output['NSA']   * 1/10              # Kg/m2
    output['SA']   = output['SA']    * 1/10              # Kg/m2
    
    output = output.fillna(-99)
    output.to_csv(out_file_path, mode='a', header=True, index=False, sep=' ')


def getting_closest_station (target_lat, target_lon, shape_location):    
    shape_location = shape_location[(shape_location.lat<target_lat+0.01) & (shape_location.lat>target_lat-0.01) & (shape_location.long<target_lon+0.01) & (shape_location.long>target_lon-0.01)].copy()
    target_point = Point(target_lon, target_lat)
    shape_location['distance'] = shape_location.apply(lambda row: target_point.distance(row['geometry']), axis=1)
    closest_point = shape_location.loc[shape_location['distance'].idxmin()]
    return closest_point.row_id, closest_point.col_id, closest_point.ID


def writing_snow_database(target_hru_revised,original_snow_path,target_snow_path): 
    
    snow=pd.read_csv(original_snow_path, skiprows= 1, sep='\s+')
    hru_data=pd.read_csv(target_hru_revised, skiprows= 1, sep='\s+')
    snow = pd.merge(snow, hru_data, left_on='name', right_on='snow', how='right')[['snow', 'fall_tmp', 'melt_tmp', 'melt_max', 'melt_min', 'tmp_lag', 'snow_h2o', 'cov50', 'snow_init']]
    snow = snow.rename(columns={'snow':'name'})    
    snow = snow.drop_duplicates(subset='name').reset_index(drop=True)
    
    snow ['fall_tmp']  = 1
    snow ['melt_tmp']  = 0.5
    snow ['melt_max']  = 4.5
    snow ['melt_min']  = 4.5
    snow ['tmp_lag']   = 1.0
    snow ['snow_h2o']  = 1.0
    snow ['cov50']     = 0.5
    snow ['snow_init'] = 0.0
    
    custom_header = 'SNODAS database for Michigan, vahid rafiei'
    
    with open(target_snow_path, 'w') as file:
        file.write(custom_header + '\n')
    snow.to_csv(target_snow_path, mode='a', sep=' ' , index=False)

def clean_and_copytree(source, target):
    if os.path.exists(target):
        shutil.rmtree(target)
    shutil.copytree(source, target)

def writing_hru_data(hrus_data, target_hru_revised):
    custom_header = 'revised hrus, vahid rafiei'
    with open(target_hru_revised, 'w') as file:
        file.write(custom_header + '\n')
    hrus_data = hrus_data.fillna('null')
    hrus_data.to_csv(target_hru_revised, mode='a', header=True, index=False, sep=' ')


def adding_statistics_to_snow_sno(BASE_PATH, LEVEL, NAME, MODEL_NAME):
    SOURCE_path = os.path.join(BASE_PATH, f'SWATplus_by_VPUID/{LEVEL}/{NAME}/{MODEL_NAME}/Scenarios/Default/TxtInOut')
    snow_sno_path = os.path.join(SOURCE_path, 'snow.sno')
    snowsno = pd.read_csv(snow_sno_path, skiprows = 1, sep='\s+')
    for i in range(len(snowsno.name)):
        
        snodas_file = pd.read_csv(os.path.join(SOURCE_path, snowsno.name[i]), skiprows=3, sep='\s+')
        start_date = datetime.datetime(2004, 1, 1)
        end_date = datetime.datetime(2023, 12, 31)
        
        date_range = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
        snodas_file['date'] = date_range
        #snodas_file = snodas_file.set_index('date')
        snodas_file = snodas_file.replace(-99,np.nan)
        
        # Add year and month columns for grouping
        snodas_file['year'] = snodas_file.date.dt.year
        snodas_file['month'] = snodas_file.date.dt.month
        
        # Group by year and month, then calculate the mean of the specified column
        monthly_means = snodas_file.groupby(['year', 'month'])['MR'].quantile(0.5)
        
        # Now calculate the mean of these monthly means for each month across all years
        mean_annual_monthly = monthly_means.groupby('month').mean()
        
        # mean_annual_monthly will now have the average of each month's mean over the years
        
        snowsno.loc[i,'MR_Jan']=round(list(mean_annual_monthly)[0], 2)
        snowsno.loc[i,'MR_Feb']=round(list(mean_annual_monthly)[1], 2)
        snowsno.loc[i,'MR_Mar']=round(list(mean_annual_monthly)[2],2)
        snowsno.loc[i,'MR_Apr']=round(list(mean_annual_monthly)[3],2)
        snowsno.loc[i,'MR_May']=round(list(mean_annual_monthly)[4],2)
        snowsno.loc[i,'MR_Jun']=round(list(mean_annual_monthly)[5],2)
        snowsno.loc[i,'MR_Jul']=round(list(mean_annual_monthly)[6],2)
        snowsno.loc[i,'MR_Aug']=round(list(mean_annual_monthly)[7],2)
        snowsno.loc[i,'MR_Sep']=round(list(mean_annual_monthly)[8],2)
        snowsno.loc[i,'MR_Oct']=round(list(mean_annual_monthly)[9],2)
        snowsno.loc[i,'MR_Nov']=round(list(mean_annual_monthly)[10],2)
        snowsno.loc[i,'MR_Dec']=round(list(mean_annual_monthly)[11],2)
        
    custom_header = 'snow data with 0.95 quantile average monthly melting rate, Vahid Rafiei 12-19-2023'    
    target_snow_path = os.path.join(SOURCE_path, 'snow.sno')    
    with open(target_snow_path, 'w') as file:
        file.write(custom_header + '\n')
    snowsno = snowsno.replace(np.nan, '-99')
    snowsno.to_csv(target_snow_path, mode='a', sep=' ' , index=False)
    
def creating_SWAT_SNODAS_files(BASE_PATH, NAME, LEVEL, MODEL_NAME, original_MODEL_NAME ):
    print(NAME)
    SOURCE = f'SWATplus_by_VPUID/{LEVEL}/{NAME}/{MODEL_NAME}/Scenarios/Default/TxtInOut/'
    out_file_path = fr'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/{LEVEL}/{NAME}/{MODEL_NAME}'
    try:
        Files_to_delete = os.listdir(out_file_path)
        Files_to_delete.remove('Scenarios')
        for file in Files_to_delete:
            os.remove(os.path.join(out_file_path,file))
    except Exception as e:
        print(e)
        

    original_source_path = os.path.join(BASE_PATH, f'SWATplus_by_VPUID/{LEVEL}/{NAME}/{original_MODEL_NAME}/Scenarios/Default/TxtInOut')

    SOURCE_path = os.path.join(BASE_PATH, SOURCE)
    
    clean_and_copytree(original_source_path, SOURCE_path)

    shutil.copy2('/data/MyDataBase/SWATGenXAppData/bin/swatplus_sno.exe', SOURCE_path)

    target_hru      =  os.path.join(BASE_PATH, SOURCE, 'hru.con'      )
    target_hru_data =  os.path.join(BASE_PATH, SOURCE, 'hru-data.hru' )
    
    hrus = pd.read_csv(target_hru, skiprows=1, sep='\s+'          )
    hrus_data = pd.read_csv(target_hru_data, skiprows=1, sep='\s+')
    
    for hru_id in range(len(hrus)):
        
        row, col, ID = getting_closest_station (hrus.lat.values[hru_id], hrus.lon.values[hru_id], SNODAS_stations)   ##### this line is very time consuming.
        #print(row, col, ID, hru_id)
        output = {}
        
        out_file_path = os.path.join(BASE_PATH,SOURCE, f'SNODAS{ID}')
        
        hrus_data.loc[hru_id,'snow'] = f'SNODAS{ID}'  

        if os.path.exists(out_file_path):
            continue
    
        variables_name = [x for x in datasets.keys()]
        
        for variable_name in variables_name:
            output[variable_name] = variables[variable_name][:, row, col]
        
        output = pd.DataFrame(output)
        now = datetime.datetime.now()
        formatted_date = now.strftime('%D')

        custom_header = f'SNODAS modeling output, created by Vahid Rafiei {formatted_date}\nlat\tlon\n{hrus.lat.values[hru_id]}\t{hrus.lon.values[hru_id]}'

        # Write the custom header
        with open(out_file_path, 'w') as file:
            file.write(custom_header + '\n')
            
        # Start date
        start_date = datetime.datetime(2004, 1, 1)
        dates = []
        # Loop to generate dates
        current_date = start_date
        while len(dates) < len(output):  # continue until the end of the final day
            dates.append(current_date)
            current_date += datetime.timedelta(days=1)
        output['date'] = dates
        saving_output(output,out_file_path)

    target_hru_revised = os.path.join(BASE_PATH,SOURCE, 'hru-data.hru')
    writing_hru_data(hrus_data, target_hru_revised )
    
    writing_snow_database(target_hru_revised,os.path.join(BASE_PATH, SOURCE, 'snow.sno'), os.path.join(BASE_PATH, SOURCE, 'snow.sno'))

LEVEL = 'huc12'
NAMES = os.listdir(f'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/{LEVEL}/')
#NAMES.remove('log.txt')

MODEL_NAME = 'SWAT_gwflow_snow_MODEL'
original_MODEL_NAME = 'SWAT_gwflow_MODEL'

for NAME in ['40802010304']:#NAMES:
    
    original_model_name_path =  os.path.join(BASE_PATH,f"SWATplus_by_VPUID/{LEVEL}/{NAME}/{original_MODEL_NAME}")
    
    if os.path.exists(original_model_name_path):
        if NAME=='4060103':
            creating_SWAT_SNODAS_files(BASE_PATH, NAME, LEVEL, MODEL_NAME, original_MODEL_NAME )
            adding_statistics_to_snow_sno(BASE_PATH, LEVEL, NAME, MODEL_NAME)
        
        if os.path.exists(fr'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc8/{NAME}/SWAT_gwflow_snow_MODEL'):
            print(f'{NAME} is already created')

            continue
            
        creating_SWAT_SNODAS_files(BASE_PATH, NAME, LEVEL, MODEL_NAME, original_MODEL_NAME )
        adding_statistics_to_snow_sno(BASE_PATH, LEVEL, NAME, MODEL_NAME)

    else:
        print('#####################################     ERROR        ERROR        ########################################')
        print(f'##################################### model {NAME} does not exists ########################################')
        print('#####################################     ERROR        ERROR        ########################################')
print('done')

In [ ]:
variable_name ='melt_rate'
output = pd.DataFrame( {'snomlt':np.array(variables[variable_name][:, row, col] )}  )

start_date = datetime.datetime(2004, 1, 1)
dates = []
# Loop to generate dates
current_date = start_date
while len(dates) < len(output):  # continue until the end of the final day
    dates.append(current_date)
    current_date += datetime.timedelta(days=1)
    
output['date'] = dates
SNODAS_station = pd.DataFrame(output)

In [ ]:
import pandas as pd

# Assuming SNODAS_station is already defined as per your code
# Convert 'date' to datetime if not already
SNODAS_station['date'] = pd.to_datetime(SNODAS_station['date'])

# Extract month from each date
SNODAS_station['month'] = SNODAS_station['date'].dt.month

# Group by month and calculate max, min, and median melt rates
SNODAS_station = SNODAS_station.dropna()
monthly_melt_stats = SNODAS_station.groupby('month')['snomlt'].agg(['max', 'min', 'median','mean'])

print(monthly_melt_stats)


In [ ]:
target_snow_path   = os.path.join(BASE_PATH, f'SWATplus_by_VPUID/{LEVEL}/{NAME}/SWAT_gwflow_snow_MODEL/snow.sno')
snow_sno = pd.read_csv(target_snow_path, skiprows=1, sep= '\s+')
snow_sno['melt_max']

In [ ]:
output

In [ ]:
output

In [ ]:
for NAME in NAMES:


In [ ]:
out_file_path